# FE 621: HW1
***
Name: Siddharth Iyer <br>
CWID: 10447455 <br>

Pledge: I pledge my honor that I have abided by the Stevens Honor System. 
***

In [17]:
# imports
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px
import plotly

import numpy as np
import pandas as pd
from scipy.stats import norm

### Problem 1

(a) Download price and implied volatility data for S&P 500 index options (SPX options). You also need the value of the S&P 500 index.
<br>
(b) Download the interest rate data from http://www.federalreserve.gov/releases/H15/Current/

In [3]:
spx = yf.Ticker("^SPX")

# SPX value current
spx_val = spx.history()["Close"]

# list of option expiry dates available for SPX
all_opt_expiry_dates = np.array(spx.options)
all_opt_expiry_dates.reshape(13,3)

array([['2022-02-11', '2022-02-14', '2022-02-16'],
       ['2022-02-18', '2022-02-22', '2022-02-23'],
       ['2022-02-25', '2022-02-28', '2022-03-02'],
       ['2022-03-04', '2022-03-07', '2022-03-11'],
       ['2022-03-18', '2022-03-25', '2022-03-31'],
       ['2022-04-14', '2022-04-29', '2022-05-20'],
       ['2022-05-31', '2022-06-17', '2022-06-30'],
       ['2022-07-15', '2022-07-29', '2022-08-19'],
       ['2022-09-16', '2022-09-30', '2022-10-21'],
       ['2022-11-18', '2022-12-16', '2022-12-30'],
       ['2023-01-20', '2023-02-17', '2023-03-17'],
       ['2023-06-16', '2023-12-15', '2024-12-20'],
       ['2025-12-19', '2026-03-20', '2026-12-18']], dtype='<U10')

In [12]:
rf_rates = {"3m":.0041, "6m":.0074, "1y":.0109}     # data collected from H15
expiry_dates = ["2022-05-20", "2022-08-19", "2023-02-17"]

In [16]:
csv_path_3m = "OptionsData\$spx-options-exp-2022-05-20, 02-11-2022.csv"
data_3m = pd.read_csv(csv_path_3m)
data_3m

,Last,Bid,Ask,Change,Volume,Open Int,IV,Last Trade,Strike,Last.1,Bid.1,Ask.1,Change.1,Volume.1,Open Int.1,IV.1,Last Trade.1
0,403.03,371.2,373.9,0.0,0,0,26.98%,1/31/2022,"4,170.00",89.30,132.9,135.0,0.0,0,0,26.38%,2/2/2022
1,411.51,367.4,370.2,0.0,0,0,26.90%,2/7/2022,"4,175.00",95.75,134.1,136.2,0.0,0,0,26.30%,2/8/2022
2,432.20,363.6,366.4,0.0,0,0,26.82%,1/31/2022,"4,180.00",123.07,135.3,137.4,0.0,0,0,26.22%,1/31/2022
3,0.00,359.8,362.6,0.0,0,0,26.73%,NaN,"4,185.00",138.90,136.5,138.6,43.5,4,0,26.14%,14:10 ET
4,415.30,356.1,358.9,0.0,0,0,26.66%,2/8/2022,"4,190.00",91.10,137.7,139.9,0.0,0,0,26.06%,2/10/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,105.62,79.1,81.0,0.0,0,0,18.80%,2/4/2022,"4,645.00",0.00,314.4,317.2,0.0,0,0,18.11%,NaN
96,98.47,77.1,78.7,0.0,0,0,18.70%,2/7/2022,"4,650.00",324.40,317.4,320.1,107.7,1,0,18.02%,15:39 ET
97,113.72,75.2,77.0,0.0,0,0,18.64%,2/4/2022,"4,655.00",0.00,320.4,323.2,0.0,0,0,17.93%,NaN
98,109.70,73.2,75.1,0.0,0,0,18.56%,2/10/2022,"4,660.00",402.48,316.2,335.9,0.0,0,0,18.00%,1/27/2022


(c) Implement the Black-Scholes option pricing formula

In [19]:
def black_scholes_options_pricing(S0, r, sig, tau, K, typ):
    """
    Returns the price of an option with user specified inputs:

        S0  (float) - initial / current price of the underlying stock \n
        r   (float) - risk free interest rate; annualized YTM of T-Bills at any maturity user wants \n
        sig (float) - volatility of underlying stock, must be in range(0,1) \n
        tau (float) - time to maturity, i.e. T-t \n
        K   (float) - strike price of option \n
        typ (str)   - specifies the type of options as either "call" or "put" \n
    
    Please don't be annyoing...only input values that make sense. \n
    Ref: https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model
    """
    sigtau = sig*np.sqrt(tau)
    d1 = 1/(sigtau) * (np.log(S0/K) + (r + np.power(sig, 2)/2) * tau)
    d2 = d1 - sigtau

    if typ == "call":
        return norm.cdf(d1)*S0 - norm.cdf(d2)*K*np.exp(-r*tau)
    elif typ == "put":
        return norm.cdf(-d2)*K*np.exp(-r*tau) - norm.cdf(-d1)*S0
    else:
        return "Not supported"
    
S0 = 4418.64
r = .0074
sig = .18
tau = 189/252
K = 4350
typ = "call"

# returns $320.49 vs $319.10 actual
black_scholes_options_pricing(S0, r, sig, tau, K, typ)

320.49475397257766

(d) Implement a function that uses either the bisection method or Newton’s method to compute the implied volatility of options.

In [33]:
def implied_volatility(mkt_price, model_params, method, err):
    """
    Returns Implied Volatility estimate based on user provided price, model parameters. \n
        mkt_price    (float): current market price of option\n
        model_params (list): [S0, r, tau, K, typ] \n
        method       (string): user will choose method of estimation (either "newton" or "bisection") \n
        err          (float):  user will provide an accuracy threshold after which program will stop execution and return \n

    """

    if method == "newton":
        pass
    elif method == "bisection":
        lower_bound = 10**-6
        upper_bound = 1

        while np.abs(upper_bound - lower_bound) > err:
            midpnt = (lower_bound + upper_bound)/2
            bs_midpnt, bs_lower = map(lambda x: black_scholes_options_pricing(model_params[0], model_params[1], x, 
                                    model_params[2], model_params[3], model_params[4]) - mkt_price, [midpnt, lower_bound])
            if bs_midpnt*bs_lower < 0:
                upper_bound = midpnt
            else:
                lower_bound = midpnt
        return np.average([lower_bound, upper_bound])

mkt_price = 317.9
err = .00001

# calculated IV 17.82% vs 23.81% actual
implied_volatility(mkt_price, [S0, r, tau, K, typ], "bisection", err)

0.17825781026840207